In [1]:
!pip install tensorflowjs #model 파일 변환을 위한 tenserflowjs 설치

     |████████████████████████████████| 77 kB 3.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf 

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers # 딥러닝 모델을 위한 라이브러리들

from sklearn.preprocessing import MultiLabelBinarizer # label 인코딩을 위한 라이브러리

from google.colab import files  #csv 파일 colab 상으로 업로드를 위함

In [3]:
seed = 10
np.random.seed(seed)

csvFile = files.upload() #데이터 파일 업로드


Saving steamEngine_dataset.csv to steamEngine_dataset.csv


In [4]:
import io
data = pd.read_csv(io.BytesIO(csvFile['steamEngine_dataset.csv']))
data.describe()

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,img_height,img_width,pose_score
count,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000,3625.000000
mean,123.718574,73.653058,0.606920,125.788554,69.978179,0.552224,127.967505,74.616081,0.536864,129.771521,67.843061,0.497974,120.267168,66.317823,0.430725,133.145517,81.448510,0.769041,117.141744,81.217201,0.770391,137.970624,93.891929,0.687224,110.790911,92.319717,0.714344,133.674526,91.731001,0.529949,120.056957,92.989141,0.496460,128.554738,133.023267,0.772574,117.016132,133.125078,0.770394,130.225307,171.061066,0.688076,117.865391,168.615332,0.690617,129.284357,199.452082,0.507291,121.665777,197.080159,0.528673,242.080828,237.753655,0.620573
std,57.228341,44.808938,0.346769,59.218618,44.182515,0.356022,58.375543,48.712331,0.344412,57.830575,43.015394,0.321495,54.065645,40.598111,0.298401,56.812274,46.516801,0.332377,53.692649,44.979769,0.327046,57.716673,60.447993,0.308821,57.675309,59.871009,0.313777,55.961574,66.921366,0.285803,54.808015,69.001980,0.280776,55.525124,62.034970,0.332646,53.409328,61.337791,0.331356,58.273733,68.380394,0.326357,51.334114,71.837590,0.317213,63.065696,92.500461,0.319307,52.788555,92.314953,0.323345,103.693524,81.781639,0.261688
min,-10.266961,-13.311756,0.000528,-7.899345,-10.793723,0.000737,-8.486717,-10.557241,0.000509,-7.927922,-15.310076,0.000386,-7.803750,-17.679083,0.000610,-7.514142,-15.425010,0.000674,-11.738182,-14.129022,0.000594,-8.927567,-6.215543,0.000537,-10.103379,-8.580183,0.000576,-9.849440,-5.287575,0.000613,-5.326314,-8.127911,0.000805,-6.494373,-6.097191,0.000802,-6.254012,-7.448987,0.000718,-10.041183,-6.990811,0.001133,-7.021356,-5.702689,0.000901,-6.753867,-0.903168,0.000942,-7.085725,-0.151686,0.000773,224.000000,224.000000,0.001182
25%,104.650563,40.616191,0.290699,106.251216,37.034905,0.221212,103.771353,38.182282,0.208045,109.285235,37.320069,0.198553,99.326125,38.070008,0.167919,111.948305,55.394295,0.678060,88.538171,54.875239,0.742457,115.423422,52.337377,0.552322,66.563086,53.291173,0.636702,112.027211,39.260535,0.325234,92.418657,47.508116,0.281846,103.304056,109.837995,0.762553,88.953761,110.081748,0.746910,99.114830,149.019780,0.447787,89.704596,131.571754,0.501901,100.737374,164.029251,0.171743,91.714089,160.380789,0.180011,224.000000,224.000000,0.543831
50%,112.981050,70.363202,0.746143,115.008597,67.215572,0.571762,113.167163,68.540700,0.583693,120.589967,66.044087,0.516822,111.242069,64.025377,0.404383,128.346296,75.133450,0.949662,105.781857,72.299741,0.934408,134.755253,91.964446,0.822708,101.039385,87.517582,0.852515,126.457822,81.791906,0.555225,109.145424,80.884282,0.515728,121.642981,117.526324,0.946756,104.189223,116.207972,0.937780,121.068177,161.850835,0.858696,104.571111,159.812515,0.852777,114.986028,192.072794,0.584640,114.743592,190.113194,0.639629,224.000000,224.000000,0.726926
75

## 데이터 셋 설명
자체 수집한 스팀엔진 사진 데이터를 PoseNet 모델을 통해 <br>
0.코   
1.왼쪽 눈  
2.오른쪽 눈  
3.왼쪽 귀  
4.오른쪽 귀  
5.왼쪽 어깨  
6.오른쪽 어깨  
7.왼쪽 팔꿈치  
8.오른쪽 팔꿈치  
9.왼쪽 손목  
10.오른쪽 손목  
11.왼쪽 골반 부위  
12.오른쪽 골반 부위  
13.왼쪽 무릎  
14.오른쪽 무릎  
15.왼쪽 발목  
16.오른쪽 발목  

각각의 x,y 좌표와 신뢰도점수, 이미지 크기와 ,전체 신뢰도 점수를 구해 데이터 셋으로 만들었습니다.  

[출처) 포즈예측 | Tensorflow Lite](https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ko)




In [5]:
for i in range(17):
  str_i = str(i)
  data[str_i+'_position_x'] = data[str_i+'_position_x'] / data['img_width']
  data[str_i+'_position_y'] = data[str_i+'_position_y'] / data['img_height']

# 이미지 크기로 각 x,y 좌표를 나누어 이미지 크기에 구애 받지 않는 데이터셋으로 변환
label = data['label']
features = data.drop(['label'],axis=1).drop(['img_width'],axis=1).drop(['img_height'],axis=1) # label과 feature 분리

In [6]:
def build_model():
  model = keras.Sequential([
                            keras.Input(shape = [len(features.keys())]),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(3, activation = 'softmax')
  ])

  model.compile(
      loss='categorical_crossentropy',
      optimizer = 'adam',
      metrics=['acc'])
  return model

In [7]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3392      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-trainable params: 0
_________________________________________________________________


In [8]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label.str.split(","))
mlb.classes_

array(['nothing', 'steamHi', 'steamready'], dtype=object)

In [9]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.3)

In [10]:
from sklearn.utils import validation
model.fit( train_x, train_y, epochs = 30 )

Epoch 1/30
80/80 [==============================] - 1s 2ms/step - loss: 0.7735 - acc: 0.7036
Epoch 2/30
80/80 [==============================] - 0s 2ms/step - loss: 0.2991 - acc: 0.9011
Epoch 3/30
80/80 [==============================] - 0s 2ms/step - loss: 0.1702 - acc: 0.9346
Epoch 4/30
80/80 [==============================] - 0s 2ms/step - loss: 0.1368 - acc: 0.9499
Epoch 5/30
80/80 [==============================] - 0s 3ms/step - loss: 0.1024 - acc: 0.9637
Epoch 6/30
80/80 [==============================] - 0s 3ms/step - loss: 0.0903 - acc: 0.9677
Epoch 7/30
80/80 [==============================] - 0s 2ms/step - loss: 0.0674 - acc: 0.9779
Epoch 8/30
80/80 [==============================] - 0s 2ms/step - loss: 0.0627 - acc: 0.9752
Epoch 9/30
80/80 [==============================] - 0s 2ms/step - loss: 0.0494 - acc: 0.9838
Epoch 10/30
80/80 [==============================] - 0s 3ms/step - loss: 0.0559 - acc: 0.9787
Epoch 11/30
80/80 [==============================] - 0s 3ms/step - lo

In [11]:
model.evaluate(test_x, test_y) #테스트 데이터로 검증
mlb.classes_

34/34 [==============================] - 0s 1ms/step - loss: 0.0548 - acc: 0.9825


array(['nothing', 'steamHi', 'steamready'], dtype=object)

In [12]:
# 테스트를 위한 코드
# csvTestFile =  files.upload() 
# test = pd.read_csv(io.BytesIO(csvTestFile['csv_downloadfile (9).csv']))
# test
# proba = model.predict(test)
# idx = np.argmax(proba)
# proba

Save Model


In [13]:
import tensorflowjs as tfjs #저장된 모델을 불러오기 위함
tfjs.converters.save_keras_model(model,artifacts_dir="./SteamModel") #tensorflow.js 사용 가능 형태로 저장
model.save('steamEngine_model.h5') #케라스 모델 저장

'/content'